In [1]:
import os
os.chdir('../')

In [2]:
from lib.readfile import ReadFile as rf
from lib.handle_null_values import handle_null_values as null
from lib.datetime_formatting import DatetimeFormatting as dfmt
from lib.split_time_variable import split_time_variable as split_t
from lib.correct_variable_types import correct_variable_types as vartype

ModuleNotFoundError: No module named 'pyspark'

In [3]:
"""Read the file"""

r= rf()
df= r.read(address="lib/data/2009.csv")

In [4]:
"""Remove column with null values"""

n= null()
variables= n.delete_var_with_null_more_than(df)

In [5]:
"""drop unwanted variables for now"""

df2= df.drop(*variables)

In [6]:
"""select important variables""" 

important_variables=['year_built', 'bedroom_count', 'bathroom_count','lot_size', 'area', 'bike_score', 'city_name', 'community_name', 'covered_parking', 'created_date', 'exterior', 'energy_source','fireplaces_count', 'floor_covering', 'garage','heating_cooling','latitude','longitude', 'property_type', 'roof', 'school_district',  'sewer', 'sold_date', 'sold_price','unfinished_sq_foot', 'topography', 'transit_score', 'was_flip', 'year_built', 'zip_code', 'county' ]

    
important_variables= list(set(important_variables))
df3= df2.select(*important_variables)



In [7]:
"""concatenate all files"""

files= ["09"]
for f_no in range(10, 20):
    files.append(f_no)    
    
    df4= r.read(address="lib/data/20"+str(10)+".csv")
    df4= df4.select(*important_variables)
    df3 = df3.union(df4)


In [8]:
"""filter one county from the dataset"""

df5= df3.filter( df3["county"].isin(["King"]))

In [9]:
"""correct time format"""

f= split_t()

#Assuming time variable is in timestamp type
df6= f.run(df5)


In [10]:
"""Correct variable types"""

v= vartype()

df7= v.run(df6)


In [13]:
p=df6.limit(500).toPandas()

numeric_columns= []
columns2= df6.columns # only take string variables

In [14]:
len(list(set(columns2)))

37

In [15]:
p['year_built']
# There is a duplication on the 

0      1959
1      1989
2      1995
3      1984
4      2006
       ... 
495    2006
496    1968
497    1975
498    2005
499    1975
Name: year_built, Length: 500, dtype: object

In [ ]:
for c in columns2:
    counter= 0
    e=[]
    float_values= []
    print(c)
    for i in range(500):
        try:
            v= float(p[c][i])
            counter= counter+1
            float_values.append(v)
        except:
            print(str(p[c][i]))
            e.append(str(p[c][i]))

    if len(e)==0:
        numeric_columns.append(c)  
    elif len(set(float_values))<5: 
        if len(set(e))<=1 and len(set(float_values))!=0:
            numeric_columns.append(c)  
    elif len(float_values)/500*100 > 95:
            numeric_columns.append(c)

In [16]:
var=find_numeric_variables_saved_as_string(df6)

NameError: name 'find_numeric_variables_saved_as_string' is not defined

In [ ]:
"""treat duplications"""
df6.select('year_built').show()

In [12]:
# train the deep learning model and put that on the training

def var_types(df7):
    categorical_variables= []
    numerical_variables=[]
    for dt in df7.dtypes:
        if dt[1]=="string":
            categorical_variables.append(dt[0])
        else:
            numerical_variables.append(dt[0])
    return categorical_variables, numerical_variables
        
categorical_variables, numerical_variables= var_types(df7)

In [44]:
"""treat null values using deep learning"""

# add an ID in the dataset
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark import SparkContext
from pyspark.sql import SparkSession
from math import *


spark_context = SparkContext.getOrCreate()
spark = SparkSession(spark_context)

windowSpec = W.orderBy(df7.columns[0])
df7= df7.withColumn("id", F.row_number().over(windowSpec))
df8= df7.createOrReplaceTempView("customer")

#use this ID to filter the data and get results.
base_query = "SELECT * FROM customer where "
total_rows= df7.count()
each_page= 500
total_pages= ceil(total_rows/each_page)

for p in range(total_pages):
    query= base_query + "id > " + str(each_page*p) + " and id< " + str(each_page*(p+1))
    df_spark= spark.sql(query)

    df_pandas= df_spark.toPandas()
    
    #pass this through all the models in the same order and run it.
    
    
    





#save the model for future use.

In [47]:
# train the deep learning model and put that on the training
categorical_variables= []
numerical_variables=[]
for dt in df7.dtypes:
    if dt[1]=="string":
        categorical_variables.append(dt[0])
    else:
        numerical_variables.append(dt[0])
        
        

In [13]:
df_train2= df7.toPandas()

In [78]:
df_train2

,address,unfinished_sq_foot,longitude,covered_parking,topography,sewer,energy_source,school_district,bike_score,sold_price,...,transit_score,latitude,sold_date_year,sold_date_month,sold_date_day,sold_datedayofweek,sold_datehour,sold_datemin,sold_datesec,id
0,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,1
1,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,2
2,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,3
3,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,4
4,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209022,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209023
209023,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209024
209024,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209025
209025,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209026


In [65]:
import datawig
#Initialize a SimpleImputer model
df_train = df7.limit(10000).toPandas()
    


In [14]:
count= 209027
train_df= df_train2[int(count*.25):int(count*.5)]
test_df= df_train2[int(count*.5):]

In [17]:
import datawig
results= {}
models={}

for c in categorical_variables: 
    var= categorical_variables.copy()
    var.remove(c)
    imputer = datawig.SimpleImputer(
        input_columns=var, # column(s) containing information about the column we want to impute
        output_column=c, # the column we'd like to impute values for
        output_path = 'lib/imputer_models2/'+str(c) # stores model data and metrics
        )
    imputer.fit(train_df=train_df, num_epochs=5)
    models[c]= imputer
    #time to test the model with entire file.  
    
#     results[c] = imputer.predict(df_pandas)
    

    

2019-10-09 11:10:52,141 [INFO]  
========== start: fit model
2019-10-09 11:10:52,142 [WARNING]  Already bound, ignoring bind()
2019-10-09 11:11:16,251 [INFO]  Epoch[0] Batch [0-994]	Speed: 662.01 samples/sec	cross-entropy=3.143252	community_name-accuracy=0.277952
2019-10-09 11:11:49,670 [INFO]  Epoch[0] Train-cross-entropy=2.153869
2019-10-09 11:11:49,672 [INFO]  Epoch[0] Train-community_name-accuracy=0.442535
2019-10-09 11:11:49,672 [INFO]  Epoch[0] Time cost=57.528
2019-10-09 11:11:49,789 [INFO]  Saved checkpoint to "lib/imputer_models2/community_name/model-0000.params"
2019-10-09 11:11:51,275 [INFO]  Epoch[0] Validation-cross-entropy=1.945110
2019-10-09 11:11:51,276 [INFO]  Epoch[0] Validation-community_name-accuracy=0.460331
2019-10-09 11:12:24,046 [INFO]  Epoch[1] Batch [0-994]	Speed: 485.48 samples/sec	cross-entropy=1.788176	community_name-accuracy=0.484108
2019-10-09 11:12:51,339 [INFO]  Epoch[1] Train-cross-entropy=1.659423
2019-10-09 11:12:51,340 [INFO]  Epoch[1] Train-communi

In [25]:
m= models[categorical_variables[0]]
xyz= m.predict(df_train2)

In [26]:
m= models[categorical_variables[1]]
xyz= m.predict(xyz)

In [72]:
results=imputer.predict(df_pandas)

In [74]:
results.to_csv("lib/clean.csv")

In [ ]:
df7.select('address').show

In [ ]:
"""encode categorical variables."""

In [ ]:
"""split data in to training and testing dataset"""

In [ ]:
p=df.limit(500).toPandas()

In [ ]:
p["ml_number"][1]